In [8]:
##- standard imports -##
import math
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import os
from typing import Optional

##- Additional Imports -##
import collections
from sklearn.cluster import KMeans
from collections import Counter
%matplotlib inline

path_to_ims = "../images"

**Canny Edge Detection**
---

Slightly optimized version of sobel filtering that guarantees certain outcomes / properties for the edge. We begin similarly to the sobel filter operation, where we need to add noise to ensure that we won't detect an edge that exists as a result of noise in the image. We'll use standard gaussian noise for our addition.

  - $\epsilon \sim \mathcal{N} (0, 1)$ : For an image normalized to $[0,1]$
  - $0 \geq \forall{x}, \forall{y} \leq 1$ in Image I : $x + \epsilon, y+ \epsilon$

**Sobel Filter**
---

Operation that measures the X and Y colour gradients of the image, and combines the two gradients into the final resultant gradient. Then we can use the following formula's to calculate the magnitude and direction of the edges:

  - $G = \sqrt{K(X)^2 + K(Y)^2}$
  - $\Theta(e) = arctan(\frac{K(Y)}{K(X)})$

**Areas of Improvement / Extra Steps**
---

Notably missing from the sobel filtering algorithm is non-maximum suppression of the output (guarantees us hard edges and less chance of noise dictating the edge). Essentially we filter and assign values to all points we aren't sure are edges, and we define thresholds for what constitutes an edge.

In [9]:
# helper functions
def print_graphs(images, bins=256):
    ### - TODO: Add x, y labels, and graph label for each plot - ###
    fig1, ax1 = plt.subplots(len(images))
    for i, x in enumerate(images):
        ax1[i].hist(x.ravel(),bins,[0,bins])

def pad_image(img: np.ndarray, kernel: list[int]) -> np.ndarray:
    # Apply padding to the image
    p_h = int((kernel[0] - 1) / 2)
    p_w = int((kernel[1] - 1) / 2)
    padded_image = np.zeros((img.shape[0] + (2 * p_h), img.shape[1] + (2 * p_w)))
    padded_image[p_h:padded_image.shape[0] - p_h, p_w:padded_image.shape[1] - p_w] = img
    return padded_image

def get_image_xgrad(outputH: np.ndarray, img: np.ndarray, padded_image: np.ndarray, kernel: list[int]):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            try:
                outputH[i][j] = abs(np.sum(np.array([1,0,-1]).T @ (np.array([1,2,1]) @ padded_image[i:i+kernel[0], j:j+kernel[1]]))) #Decomposability of sobel filters
            except ValueError: ###was useful in border handling
                print(i,j)
    return outputH

def get_image_ygrad(outputV: np.ndarray, img: np.ndarray, padded_image: np.ndarray, kernel: list[int]):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            try:
                outputV[i][j] = abs(np.sum(np.array([1,2,1]).T @ (np.array([-1,0,1]) @ padded_image[i:i+kernel[0], j:j+kernel[1]]))) #Decomposability of sobel filters
            except ValueError: ###was useful in border handling
                print(i,j)
    return outputV
 
#sobel function
def Sobel_Filter(img, kernel=[3,3]): #This function will kind of just assume 3x3
    outputH = np.zeros(img.shape)
    outputV = np.zeros(img.shape) #We have 3 outputs, this function will display histograms for Horizontal and vertical
    output = np.zeros(img.shape)
    
    ###Smooth images###
    img = cv.GaussianBlur(img, (kernel[0],kernel[0]), np.std(img)/8) # The amount of gaussian blur can directly determine the sharpness of the sobel filter edge
    
    ###Unsure of if this is useful for edge detection but since we apply the same convolution algorithm:###
    ###Convert to padded image###
    padded_image = pad_image(img, kernel)
    
    #Convolve with filters Horizontal
    outputH = get_image_xgrad(outputH, img, padded_image, kernel)
    outputV = get_image_ygrad(outputV, img, padded_image, kernel)
    
    assert(outputH.shape == img.shape and outputV.shape == img.shape) #quality of life
    
    ###Normalize###
    outputHN = (outputH-np.min(outputH)) / (np.max(outputH)-np.min(outputH))
    outputVN = (outputV-np.min(outputV)) / (np.max(outputV)-np.min(outputV))
    
    print('Horizontal Range: ' + str((np.min(outputHN), np.max(outputHN))))
    print('Vertical Range: ' + str((np.min(outputVN), np.max(outputVN))))
    
    ###Merge horizontal and vertical###
    output = np.sqrt(outputH**2 + outputV**2)
    
    outputN = (output-np.min(output)) / (np.max(output)-np.min(output))
    angles = np.degrees(np.arctan(outputVN / outputHN))
    
    return [outputHN*255, outputVN*255, outputN*255, angles] # scale back to image output

**Thresholding and Non-Maximum Suppression**
---

These algorithms depend on normalizing the thresholds to within the bounds of the image, and assigning weak and strong pixel values for edges, that way if we decide on an edge we can give it a pixel value depending on how confident we are that pixel i, j is an edge.

Additionally we can assert to ourselves that a weak pixel on an edge is part of the edge by checking for other strong pixels around the weak pixel. This will aim to transform as many of our weak pixels into strong pixels. If we find a strong pixel on a weak edge, assign the pixel to the strong edge value.

In [10]:
# Non-maximum suppression algorithm
def non_max_suppression(img, degrees):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = degrees * 180. / np.pi
    angle[angle < 0] += 180

    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0

            except IndexError as e:
                pass
    
    return Z

def threshold(img, high_threshold: Optional[float]=0.05, low_threshold: Optional[float]=0.01, strong_pix: Optional[float]=255., weak_pix: Optional[float] = 75):
    # apply thresholding to img
    high_threshold = high_threshold;
    low_threshold = low_threshold;

    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)

    weak = np.int32(weak_pix)
    strong = np.int32(strong_pix)

    strong_i, strong_j = np.where(img >= high_threshold)
    zeros_i, zeros_j = np.where(img < low_threshold) #apply if needed

    weak_i, weak_j = np.where((img <= high_threshold) & (img >= low_threshold))

    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak

    return (res)

def hysterisis(img: np.ndarray, strong_pix: Optional[float]=255., weak_pix: Optional[float]=75):
    M, N = img.shape
    weak = np.int32(weak_pix)
    strong = np.int32(strong_pix)  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img


def canny_edge(image: np.ndarray, with_hysterisis: Optional[bool]=False, **kwargs):
    horizontal, vertical, neutral, degrees = Sobel_Filter(image)

    suppressed = non_max_suppression(neutral, degrees)
    threshold_applied = threshold(suppressed, **kwargs)
    if with_hysterisis:
        try:
            return hysterisis(threshold_applied, **kwargs)
        except:
            return hysterisis(threshold_applied, strong_pix=kwargs.get("strong_pix"), weak_pix=kwargs.get("weak_pix"))
    
    else:
        return threshold_applied

**Without Hysterisis**
---

In [11]:
im2_ = cv.imread(f"{path_to_ims}/image1.jpg")
# convert to grayscale
gim2 = cv.cvtColor(im2_, cv.COLOR_BGR2GRAY)

canny_edge_im = canny_edge(gim2, weak_pix=200, strong_pix=255)

im2 = np.zeros_like(im2_)
im2[:,:,0] = canny_edge_im # To display the image correctly you'll need to send the grey channel to each of the three image channels
im2[:,:,1] = canny_edge_im
im2[:,:,2] = canny_edge_im
plt.imshow(im2)

In [5]:
plt.imshow(canny_edge_im)

**With Hysterisis**
---

In [6]:
im2_ = cv.imread(f"{path_to_ims}/image1.jpg")
# convert to grayscale
gim2 = cv.cvtColor(im2_, cv.COLOR_BGR2GRAY)

canny_edge_im = canny_edge(gim2, with_hysterisis=True, weak_pix=200, strong_pix=255)

im2 = np.zeros_like(im2_)
im2[:,:,0] = canny_edge_im # To display the image correctly you'll need to send the grey channel to each of the three image channels
im2[:,:,1] = canny_edge_im
im2[:,:,2] = canny_edge_im
plt.imshow(im2)

Horizontal Range: (0.0, 1.0)
Vertical Range: (0.0, 1.0)


/tmp/ipykernel_37601/263653068.py:64: RuntimeWarning: divide by zero encountered in divide
  angles = np.degrees(np.arctan(outputVN / outputHN))
/tmp/ipykernel_37601/263653068.py:64: RuntimeWarning: invalid value encountered in divide
  angles = np.degrees(np.arctan(outputVN / outputHN))


In [7]:
plt.imshow(canny_edge_im)